In [0]:
# # Let's make sure the kaggle.json file is present.
# !ls -lha kaggle.json
# # Next, install the Kaggle API client.
# !pip install -q kaggle
# # The Kaggle API client expects this file to be in ~/.kaggle,
# # so move it there.
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# # This permissions change avoids a warning on Kaggle tool startup.
# !chmod 600 ~/.kaggle/kaggle.json
# # List available datasets.
# !kaggle datasets list
# # Copy the stackoverflow data set locally.
# !kaggle datasets download -d  news-headlines-dataset-for-sarcasm-detection
# # !unzip /content/barcelona-data-sets.zip

In [0]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

#Visualising Data

In [0]:
!unzip /content/news-headlines-dataset-for-sarcasm-detection.zip

Archive:  /content/news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  


In [0]:



data = pd.read_json('/content/Sarcasm_Headlines_Dataset.json',lines = True)

data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [0]:
data['headline'][3]

'boehner just wants wife to listen, not come up with alternative debt-reduction ideas'

In [0]:
print('1 - is sarcastic and 0- not sarcastic')

1 - is sarcastic and 0- not sarcastic


In [0]:
data.describe()

,is_sarcastic
count,26709.000000
mean,0.438953
std,0.496269
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [0]:
print('getting traning and testing data')

length = data['is_sarcastic'].shape[0]

train_data = data['headline'][0:int(length*0.8)].values

test_data = data['headline'][int(length*0.8):].values

print(train_data.shape , length * 0.8)
print(test_data.shape,length * 0.2)

train_label = data['is_sarcastic'][0:int(length*0.8)].values
test_label = data['is_sarcastic'][int(length * 0.8):].values




getting traning and testing data
(21367,) 21367.2
(5342,) 5341.8


In [0]:
train_data[0:50]

array(["former versace store clerk sues over secret 'black code' for minority shoppers",
       "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
       "mom starting to fear son's web series closest thing she will have to grandchild",
       'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
       'j.k. rowling wishes snape happy birthday in the most magical way',
       "advancing the world's women",
       'the fascinating case for eating lab-grown meat',
       'this ceo will send your kids to school, if you work for his company',
       'top snake handler leaves sinking huckabee campaign',
       "friday's morning email: inside trump's presser for the ages",
       'airline passengers tackle man who rushes cockpit in bomb threat',
       'facebook reportedly working on healthcare features and apps',
       "north korea praises trump and urges us voters to reject 'dull hillary'",
       "actually, cnn's jef

#Preprocessing Data

##TODO:

<pre>
 2>remove all the pancuations

 4>vectorize / embedding
 </pre>

In [0]:
from keras.preprocessing import text

Using TensorFlow backend.


In [0]:

print('converting words to tokens ')

print(train_data[0:5])

x = []
for i in train_data:
  
  x.append(text.text_to_word_sequence(i,filters = '\," \'!.'))

print(x[0:2])




converting words to tokens 
["former versace store clerk sues over secret 'black code' for minority shoppers"
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse"
 "mom starting to fear son's web series closest thing she will have to grandchild"
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas'
 'j.k. rowling wishes snape happy birthday in the most magical way']
[['former', 'versace', 'store', 'clerk', 'sues', 'over', 'secret', 'black', 'code', 'for', 'minority', 'shoppers'], ['the', 'roseanne', 'revival', 'catches', 'up', 'to', 'our', 'thorny', 'political', 'mood', 'for', 'better', 'and', 'worse']]


In [0]:
print('finding the padding length')

m = len(x[0])
sen = []
for i in x[1:]:
  
  if len(i) > m:
      m = len(i) 
      sen = i 
    
print(m,sen)

padding_size = m

print('padding')

x2 = []

for i in x:
  
  
  if len(i) < padding_size:
      
      for j in range(padding_size - len(i)):
        
        i.append('0')
        
 

finding the padding length
39 ['elmore', 'leonard', 'modern', 'prose', 'master', 'noted', 'for', 'his', 'terse', 'prose', 'style', 'and', 'for', 'writing', 'about', 'things', 'perfectly', 'and', 'succinctly', 'with', 'a', 'remarkable', 'economy', 'of', 'words', 'unfortunately', 'and', 'sadly', 'expired', 'this', 'gloomy', 'tuesday', 'at', 'the', 'age', 'of', '87', 'years', 'old']
padding


#Embedings

In [0]:
from gensim import corpora,models,similarities


/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [0]:
model_g = models.Word2Vec(x,min_count = 1,size = 300)

In [0]:
#embedding for word 
print(x[0][2])
model_g.most_similar(x[0][2])  



store


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hope', 0.9993831515312195),
 ('totally', 0.9991444945335388),
 ('restaurant', 0.9990803599357605),
 ('free', 0.9990619421005249),
 ('still', 0.9989666938781738),
 ('mall', 0.9988510608673096),
 ('left', 0.9988348484039307),
 ('food', 0.9988080263137817),
 ('california', 0.9987903833389282),
 ('god', 0.9987727999687195)]

#Encod

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('nearly', 0.9993244409561157),
 ('david', 0.9992198944091797),
 ('protesters', 0.9992049932479858),
 ('board', 0.9991206526756287),
 ('group', 0.9991177320480347),
 ('leader', 0.9990381002426147),
 ('male', 0.999008297920227),
 ('alert', 0.998997688293457),
 ('showing', 0.9989637136459351),
 ('single', 0.9989575147628784)]

In [0]:

from keras.layers import Dense,LSTM,Embedding    
from keras.models import Sequential   
import numpy as np
from sklearn.model_selection import train_test_split

model = Sequential()
model.add(Embedding(50,64,input_length = 39))
model.add(LSTM(1,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])




In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 39, 64)            64000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 1)                 264       
Total params: 64,264
Trainable params: 64,264
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.get_input_shape_at(0)

(None, 39)

In [0]:
model.fit(x = train_data,y = train_label)

Epoch 1/1
21367/21367 [==============================] - 16s 731us/step - loss: 0.6874 - acc: 0.5593


##SELECTING EMBEDDING MODEL FROM HUB

#https://tfhub.dev/google/nnlm-en-dim128/1

In [0]:
import time

def sess(embeddings): 
  s = time.time()
  with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      sess.run(tf.tables_initializer())

      ress =sess.run(embeddings)
  print(time.time() - s)  

In [0]:
# !pip install "tensorflow_hub==0.4.0"
# !pip install "tf-nightly"
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
embeddings = embed(["cat dad  ds das d a d ", "dog  da sd asd asdas d as d"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:21:32.276868 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
sess(embeddings)

1.9271552562713623


In [0]:
ress.shape

(2, 128)

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
    inputs={
        "tokens": tokens_input,
        "sequence_len": tokens_length
    },
    signature="tokens",
    as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:25:41.617915 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
sess(embeddings)

4.031406402587891


In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]
sess(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:26:32.146115 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


3.9208099842071533


In [0]:
embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

sess(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:27:36.526976 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


4.226448059082031


In [0]:
embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])



sess(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:29:11.036698 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


5.5092973709106445


#going with https://tfhub.dev/google/nnlm-en-dim128/1"

In [0]:
# !pip install "tensorflow_hub==0.4.0"
# !pip install "tf-nightly"

embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
embeddings = embed(["cat dad  ds das d a d ", "dog  da sd asd asdas d as d"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0426 17:30:24.948648 140154410268544 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
import time

emb = []

s = time.time()
with tf.Graph().as_default():
  embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
  messages = tf.placeholder(dtype=tf.string, shape=[None])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    
    sent = embed(messages)
        
    w =  0
    for i in train_data:
        w +=1
#         print('counter',w)
        
        
        
        emb.append(sess.run(sent,feed_dict = {messages:[i]}))
        
        
        
print(time.time() - s)  

In [0]:
emb[0].shape

(1, 128)

In [0]:
train_data = np.array(emb)
print(train_data.shape)

(21367, 1, 128)


In [0]:

from keras.layers import Dense,LSTM,Embedding    
from keras.models import Sequential   
import numpy as np
from sklearn.model_selection import train_test_split

model = Sequential()
model.add(LSTM(64,input_shape = (1,128),return_sequences = True,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(128,kernel_initializer = 'glorot_normal',return_sequences = True,
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(1,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])




In [0]:
model.fit?

In [0]:
model.fit(x = train_data,y = train_label,epochs = 10,batch_size = 24)

Epoch 1/10
21367/21367 [==============================] - 49s 2ms/step - loss: 0.5183 - acc: 0.7631
Epoch 2/10
21367/21367 [==============================] - 7s 331us/step - loss: 0.4708 - acc: 0.7954
Epoch 3/10
21367/21367 [==============================] - 7s 336us/step - loss: 0.4624 - acc: 0.8012
Epoch 4/10
21367/21367 [==============================] - 7s 343us/step - loss: 0.4574 - acc: 0.8038
Epoch 5/10
21367/21367 [==============================] - 7s 332us/step - loss: 0.4552 - acc: 0.8060
Epoch 6/10
21367/21367 [==============================] - 7s 333us/step - loss: 0.4455 - acc: 0.8132
Epoch 7/10
21367/21367 [==============================] - 7s 330us/step - loss: 0.4382 - acc: 0.8192
Epoch 8/10
21367/21367 [==============================] - 7s 333us/step - loss: 0.4275 - acc: 0.8264
Epoch 9/10
21367/21367 [==============================] - 7s 323us/step - loss: 0.4180 - acc: 0.8355
Epoch 10/10
21367/21367 [==============================] - 7s 320us/step - loss: 0.4176 - ac

In [0]:
for i in range(100):
  
  
  print(model.predict(np.expand_dims(train_data[i],axis = 0)))

#EMBEDDING 2 (word2Vec)

In [0]:
import time

emb = []

s = time.time()
with tf.Graph().as_default():
  embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
  messages = tf.placeholder(dtype=tf.string, shape=[None])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    
    sent = embed(messages)
        
    w =  0
    for i in train_data:
        w +=1
#         print('counter',w)
        
        
        
        emb.append(sess.run(sent,feed_dict = {messages:[i]}))
        
        
        
print(time.time() - s)  




W0427 02:17:50.735309 139717700630400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


26.76400876045227


In [0]:
train_data = np.array(emb)
print(train_data.shape)

(21367, 1, 250)


In [0]:

from keras.layers import Dense,LSTM,Embedding    
from keras.models import Sequential   
import numpy as np
from sklearn.model_selection import train_test_split

model = Sequential()
model.add(LSTM(64,input_shape = (1,250),return_sequences = True,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(128,kernel_initializer = 'glorot_normal',return_sequences = True,
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(1,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])




In [0]:
model.fit(x = train_data,y = train_label,epochs = 10,batch_size = 24)


W0427 02:19:40.597457 139717700630400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/10
21367/21367 [==============================] - 15s 682us/step - loss: 0.5891 - acc: 0.6963
Epoch 2/10
21367/21367 [==============================] - 12s 554us/step - loss: 0.5354 - acc: 0.7467
Epoch 3/10
21367/21367 [==============================] - 12s 549us/step - loss: 0.5226 - acc: 0.7573
Epoch 4/10
21367/21367 [==============================] - 12s 544us/step - loss: 0.5044 - acc: 0.7695
Epoch 5/10
21367/21367 [==============================] - 12s 548us/step - loss: 0.4980 - acc: 0.7744
Epoch 6/10
21367/21367 [==============================] - 12s 548us/step - loss: 0.4897 - acc: 0.7810
Epoch 7/10
21367/21367 [==============================] - 12s 551us/step - loss: 0.4785 - acc: 0.7907
Epoch 8/10
21367/21367 [==============================] - 12s 551us/step - loss: 0.4710 - acc: 0.7928
Epoch 9/10
21367/21367 [==============================] - 12s 545us/step - loss: 0.4703 - acc: 0.7954
Epoch 10/10
21367/21367 [==============================] - 12s 543us/step - loss: 

In [0]:
for i in range(100):
  
  
  print(np.round(model.predict(np.expand_dims(train_data[i],axis = 0))))

In [0]:
model.evaluate(train_data,train_label)

21367/21367 [==============================] - 3s 122us/step


[0.4470132531610153, 0.8115318013564267]

#using Universal sentence encoder

In [0]:
print('getting traning and testing data')

length = data['is_sarcastic'].shape[0]

train_data = data['headline'][0:int(length*0.8)].values

test_data = data['headline'][int(length*0.8):].values

print(train_data.shape , length * 0.8)
print(test_data.shape,length * 0.2)

train_label = data['is_sarcastic'][0:int(length*0.8)].values
test_label = data['is_sarcastic'][int(length * 0.8):].values

getting traning and testing data
(21367,) 21367.2
(5342,) 5341.8


In [0]:
import time

emb = []

s = time.time()
with tf.Graph().as_default():
  embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
  messages = tf.placeholder(dtype=tf.string, shape=[None])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    
    sent = embed(messages)
        
    w =  0
    for i in train_data:
        w +=1
#         print('counter',w)
        
        
        
        emb.append(sess.run(sent,feed_dict = {messages:[i]}))
        
        
        
print(time.time() - s)  


57.517850160598755


In [0]:
train_data = np.array(emb)
print(train_data.shape)

(21367, 1, 512)


In [0]:

from keras.layers import Dense,LSTM,Embedding    
from keras.models import Sequential   
import numpy as np
from sklearn.model_selection import train_test_split

model = Sequential()
model.add(LSTM(64,input_shape = (1,512),return_sequences = True,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(128,kernel_initializer = 'glorot_normal',return_sequences = True,
               recurrent_initializer = 'glorot_normal',activation = 'relu'))
model.add(LSTM(1,kernel_initializer = 'glorot_normal',
               recurrent_initializer = 'glorot_normal',activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])



In [0]:
model.fit(x = train_data,y = train_label,epochs = 10,batch_size = 24)

Epoch 1/10
21367/21367 [==============================] - 17s 790us/step - loss: 0.5571 - acc: 0.7261
Epoch 2/10
21367/21367 [==============================] - 14s 646us/step - loss: 0.4933 - acc: 0.7737
Epoch 3/10
21367/21367 [==============================] - 14s 647us/step - loss: 0.4739 - acc: 0.7910
Epoch 4/10
21367/21367 [==============================] - 14s 643us/step - loss: 0.4627 - acc: 0.7998
Epoch 5/10
21367/21367 [==============================] - 14s 646us/step - loss: 0.4494 - acc: 0.8079
Epoch 6/10
21367/21367 [==============================] - 14s 643us/step - loss: 0.4394 - acc: 0.8154
Epoch 7/10
21367/21367 [==============================] - 14s 648us/step - loss: 0.4329 - acc: 0.8194
Epoch 8/10
21367/21367 [==============================] - 14s 648us/step - loss: 0.4231 - acc: 0.8280
Epoch 9/10
21367/21367 [==============================] - 14s 645us/step - loss: 0.4206 - acc: 0.8272
Epoch 10/10
21367/21367 [==============================] - 14s 645us/step - loss: 

In [0]:
for i in range(100):
  
  
  print(np.round(model.predict(np.expand_dims(train_data[i],axis = 0))))

In [0]:
model.evaluate(train_data,train_label)

21367/21367 [==============================] - 3s 129us/step


[0.39555844720643935, 0.8479898909338123]